In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import mediapipe as mp
from PIL import Image

## 1. Import Required Libraries

# Upper Body Skeleton Reconstruction from Apple Vision Pro Data

This notebook implements methods to reconstruct upper body skeletal joints from partial Apple Vision Pro tracking data (head and hands).

**Objectives:**
- Load and visualize existing device pose and hand tracking data
- Implement anthropometric-based joint estimation
- Develop inverse kinematics for elbow positioning
- Visualize the complete upper body skeleton

## 2. Load Vision Pro Data

In [ ]:
# Load device pose data (head tracking)
device_data = pd.read_csv('Prototype/Data/device_pose_data .csv')
print(f"Device data shape: {device_data.shape}")
print(f"Columns: {device_data.columns.tolist()}")
device_data.head()

In [ ]:
# Load hand tracking data
hand_data = pd.read_csv('Prototype/Data/hand_data_pivoted.csv')
print(f"Hand data shape: {hand_data.shape}")
print(f"Number of frames: {len(hand_data)}")
print(f"\nSample columns (first 10): {hand_data.columns.tolist()[:10]}")
hand_data.head()

## 3. Extract Key Joint Data

Extract the important joints we have: head, wrists, and forearm positions.

In [ ]:
def extract_joint_positions(hand_df, joint_name, chirality='right'):
    """Extract position and orientation for a specific joint."""
    filtered = hand_df[hand_df['chirality'] == chirality]
    
    position = filtered[[f'{joint_name}_px', f'{joint_name}_py', f'{joint_name}_pz']].values
    orientation = filtered[[f'{joint_name}_qx', f'{joint_name}_qy', f'{joint_name}_qz', f'{joint_name}_qw']].values
    
    return position, orientation

# Extract wrist and forearm positions for both hands
right_wrist_pos, right_wrist_ori = extract_joint_positions(hand_data, 'forearmWrist', 'right')
left_wrist_pos, left_wrist_ori = extract_joint_positions(hand_data, 'forearmWrist', 'left')

right_forearm_pos, right_forearm_ori = extract_joint_positions(hand_data, 'forearmArm', 'right')
left_forearm_pos, left_forearm_ori = extract_joint_positions(hand_data, 'forearmArm', 'left')

print(f"Right wrist positions: {right_wrist_pos.shape}")
print(f"Left wrist positions: {left_wrist_pos.shape}")
print(f"Right forearm positions: {right_forearm_pos.shape}")
print(f"Left forearm positions: {left_forearm_pos.shape}")

In [ ]:
# Extract head positions
head_positions = device_data[['x', 'y', 'z']].values
head_orientations = device_data[['qx', 'qy', 'qz', 'qw']].values

print(f"Head positions: {head_positions.shape}")
print(f"Sample head position: {head_positions[0]}")
print(f"Sample head orientation (quaternion): {head_orientations[0]}")

## 4. Visualize Existing Data

Visualize the raw data we have: head position and hand joints.

In [ ]:
# Visualize a single frame of existing data
frame_idx = 100  # Choose a frame to visualize

fig = go.Figure()

# Add head position
fig.add_trace(go.Scatter3d(
    x=[head_positions[frame_idx, 0]],
    y=[head_positions[frame_idx, 1]],
    z=[head_positions[frame_idx, 2]],
    mode='markers',
    marker=dict(size=10, color='red'),
    name='Head'
))

# Add right hand joints
fig.add_trace(go.Scatter3d(
    x=[right_wrist_pos[frame_idx, 0], right_forearm_pos[frame_idx, 0]],
    y=[right_wrist_pos[frame_idx, 1], right_forearm_pos[frame_idx, 1]],
    z=[right_wrist_pos[frame_idx, 2], right_forearm_pos[frame_idx, 2]],
    mode='markers+lines',
    marker=dict(size=7, color='blue'),
    line=dict(color='blue', width=3),
    name='Right Arm'
))

# Add left hand joints
fig.add_trace(go.Scatter3d(
    x=[left_wrist_pos[frame_idx, 0], left_forearm_pos[frame_idx, 0]],
    y=[left_wrist_pos[frame_idx, 1], left_forearm_pos[frame_idx, 1]],
    z=[left_wrist_pos[frame_idx, 2], left_forearm_pos[frame_idx, 2]],
    mode='markers+lines',
    marker=dict(size=7, color='green'),
    line=dict(color='green', width=3),
    name='Left Arm'
))

fig.update_layout(
    title=f'Existing Vision Pro Data (Frame {frame_idx})',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    width=800,
    height=600
)

fig.show()

## 5. Define Anthropometric Parameters

Set up body proportions based on user height.

In [ ]:
class AnthropometricModel:
    """
    Anthropometric body proportions based on user height.
    Standard ratios from biomechanical literature.
    """
    def __init__(self, height_m=1.75):
        self.height = height_m
        
        # Body segment lengths as percentages of height
        self.neck_length = 0.05 * height_m          # 5% of height
        self.shoulder_width = 0.25 * height_m       # 25% of height
        self.upper_arm_length = 0.18 * height_m     # 18% of height
        self.forearm_length = 0.16 * height_m       # 16% of height
        self.spine_length = 0.30 * height_m         # 30% of height (total)
        
        # Spine segments (divide total spine into 3 segments)
        self.upper_spine_length = 0.10 * height_m
        self.mid_spine_length = 0.10 * height_m
        self.lower_spine_length = 0.10 * height_m
        
    def __repr__(self):
        return f"""AnthropometricModel(height={self.height}m)
  Neck: {self.neck_length:.3f}m
  Shoulder width: {self.shoulder_width:.3f}m
  Upper arm: {self.upper_arm_length:.3f}m
  Forearm: {self.forearm_length:.3f}m
  Spine: {self.spine_length:.3f}m"""

# Create model (user can input their height)
USER_HEIGHT = 1.75  # meters (adjust as needed)
body_model = AnthropometricModel(USER_HEIGHT)
print(body_model)

## 6. Implement Joint Estimation Methods

Methods to estimate missing joints (neck, shoulders, spine, elbows).

In [ ]:
def quaternion_to_rotation_matrix(q):
    """Convert quaternion [qx, qy, qz, qw] to rotation matrix."""
    qx, qy, qz, qw = q
    return np.array([
        [1 - 2*(qy**2 + qz**2), 2*(qx*qy - qz*qw), 2*(qx*qz + qy*qw)],
        [2*(qx*qy + qz*qw), 1 - 2*(qx**2 + qz**2), 2*(qy*qz - qx*qw)],
        [2*(qx*qz - qy*qw), 2*(qy*qz + qx*qw), 1 - 2*(qx**2 + qy**2)]
    ])

def get_down_vector(head_quaternion):
    """Get the downward direction vector from head orientation."""
    rot_matrix = quaternion_to_rotation_matrix(head_quaternion)
    # Down vector is negative Y in local frame
    down_vector = -rot_matrix[:, 1]
    return down_vector / np.linalg.norm(down_vector)

# Test the functions
test_quat = head_orientations[100]
down = get_down_vector(test_quat)
print(f"Test head quaternion: {test_quat}")
print(f"Down vector: {down}")

In [ ]:
def estimate_neck_position(head_pos, head_quat, body_model):
    """Estimate neck position as offset below head."""
    down_vector = get_down_vector(head_quat)
    neck_pos = head_pos + down_vector * body_model.neck_length
    return neck_pos

def estimate_spine_positions(neck_pos, head_quat, body_model):
    """Estimate upper, mid, and lower spine positions."""
    down_vector = get_down_vector(head_quat)
    
    upper_spine = neck_pos + down_vector * body_model.upper_spine_length
    mid_spine = upper_spine + down_vector * body_model.mid_spine_length
    lower_spine = mid_spine + down_vector * body_model.lower_spine_length
    
    return upper_spine, mid_spine, lower_spine

# Test on a single frame
test_frame = 100
test_neck = estimate_neck_position(head_positions[test_frame], head_orientations[test_frame], body_model)
test_spines = estimate_spine_positions(test_neck, head_orientations[test_frame], body_model)

print(f"Head position: {head_positions[test_frame]}")
print(f"Estimated neck: {test_neck}")
print(f"Estimated upper spine: {test_spines[0]}")
print(f"Estimated mid spine: {test_spines[1]}")
print(f"Estimated lower spine: {test_spines[2]}")

In [ ]:
def estimate_shoulder_positions(upper_spine_pos, head_quat, body_model):
    """Estimate left and right shoulder positions."""
    rot_matrix = quaternion_to_rotation_matrix(head_quat)
    # Right vector in local frame
    right_vector = rot_matrix[:, 0]
    right_vector = right_vector / np.linalg.norm(right_vector)
    
    # Shoulders are at shoulder width apart
    left_shoulder = upper_spine_pos - right_vector * (body_model.shoulder_width / 2)
    right_shoulder = upper_spine_pos + right_vector * (body_model.shoulder_width / 2)
    
    return left_shoulder, right_shoulder

# Test shoulder estimation
test_upper_spine = test_spines[0]
test_left_shoulder, test_right_shoulder = estimate_shoulder_positions(
    test_upper_spine, head_orientations[test_frame], body_model
)

print(f"Upper spine: {test_upper_spine}")
print(f"Left shoulder: {test_left_shoulder}")
print(f"Right shoulder: {test_right_shoulder}")
print(f"Shoulder distance: {np.linalg.norm(test_right_shoulder - test_left_shoulder):.3f}m")

## 7. Implement Elbow IK with Three-Point Constraint

Use shoulder (estimated), forearm (known), and wrist (known) to solve for elbow position.

In [ ]:
def estimate_elbow_position(shoulder_pos, forearm_pos, wrist_pos, upper_arm_length):
    """
    Estimate elbow position using three constraint points.
    
    Strategy:
    - Elbow should be between shoulder and forearm joint
    - Distance from shoulder to elbow = upper_arm_length
    - Elbow positioned along the shoulder-to-forearm direction
    """
    # Direction from shoulder to forearm
    shoulder_to_forearm = forearm_pos - shoulder_pos
    distance_to_forearm = np.linalg.norm(shoulder_to_forearm)
    
    if distance_to_forearm < upper_arm_length:
        # Forearm is closer than upper arm length - place elbow proportionally
        direction = shoulder_to_forearm / distance_to_forearm
        elbow_pos = shoulder_pos + direction * upper_arm_length
    else:
        # Normal case - place elbow at upper arm length along the direction
        direction = shoulder_to_forearm / distance_to_forearm
        elbow_pos = shoulder_pos + direction * upper_arm_length
    
    return elbow_pos

# Test elbow estimation
test_right_elbow = estimate_elbow_position(
    test_right_shoulder,
    right_forearm_pos[test_frame],
    right_wrist_pos[test_frame],
    body_model.upper_arm_length
)

test_left_elbow = estimate_elbow_position(
    test_left_shoulder,
    left_forearm_pos[test_frame],
    left_wrist_pos[test_frame],
    body_model.upper_arm_length
)

print(f"Right elbow: {test_right_elbow}")
print(f"Distance shoulder to elbow: {np.linalg.norm(test_right_elbow - test_right_shoulder):.3f}m")
print(f"\nLeft elbow: {test_left_elbow}")
print(f"Distance shoulder to elbow: {np.linalg.norm(test_left_elbow - test_left_shoulder):.3f}m")

## 8. Complete Skeleton Reconstruction

Combine all estimation methods into a complete skeleton reconstruction function.

In [ ]:
class UpperBodySkeleton:
    """Complete upper body skeleton with all joints."""
    def __init__(self):
        self.head = None
        self.neck = None
        self.upper_spine = None
        self.mid_spine = None
        self.lower_spine = None
        self.left_shoulder = None
        self.right_shoulder = None
        self.left_elbow = None
        self.right_elbow = None
        self.left_wrist = None
        self.right_wrist = None
        self.left_forearm = None
        self.right_forearm = None
    
    def get_all_joints(self):
        """Return dictionary of all joint positions."""
        return {
            'head': self.head,
            'neck': self.neck,
            'upper_spine': self.upper_spine,
            'mid_spine': self.mid_spine,
            'lower_spine': self.lower_spine,
            'left_shoulder': self.left_shoulder,
            'right_shoulder': self.right_shoulder,
            'left_elbow': self.left_elbow,
            'right_elbow': self.right_elbow,
            'left_wrist': self.left_wrist,
            'right_wrist': self.right_wrist,
        }

def reconstruct_skeleton(frame_idx, head_pos, head_quat, 
                         left_wrist, right_wrist,
                         left_forearm, right_forearm,
                         body_model):
    """
    Reconstruct complete upper body skeleton for a single frame.
    """
    skeleton = UpperBodySkeleton()
    
    # Known joints
    skeleton.head = head_pos
    skeleton.left_wrist = left_wrist
    skeleton.right_wrist = right_wrist
    skeleton.left_forearm = left_forearm
    skeleton.right_forearm = right_forearm
    
    # Estimate hardcoded joints
    skeleton.neck = estimate_neck_position(head_pos, head_quat, body_model)
    upper, mid, lower = estimate_spine_positions(skeleton.neck, head_quat, body_model)
    skeleton.upper_spine = upper
    skeleton.mid_spine = mid
    skeleton.lower_spine = lower
    
    # Estimate shoulders
    skeleton.left_shoulder, skeleton.right_shoulder = estimate_shoulder_positions(
        skeleton.upper_spine, head_quat, body_model
    )
    
    # Estimate elbows using IK
    skeleton.left_elbow = estimate_elbow_position(
        skeleton.left_shoulder, left_forearm, left_wrist, body_model.upper_arm_length
    )
    skeleton.right_elbow = estimate_elbow_position(
        skeleton.right_shoulder, right_forearm, right_wrist, body_model.upper_arm_length
    )
    
    return skeleton

# Test complete reconstruction
test_skeleton = reconstruct_skeleton(
    test_frame,
    head_positions[test_frame],
    head_orientations[test_frame],
    left_wrist_pos[test_frame],
    right_wrist_pos[test_frame],
    left_forearm_pos[test_frame],
    right_forearm_pos[test_frame],
    body_model
)

print("Reconstructed skeleton joints:")
for joint_name, joint_pos in test_skeleton.get_all_joints().items():
    if joint_pos is not None:
        print(f"  {joint_name}: {joint_pos}")

## 9. Visualize Complete Reconstructed Skeleton

Visualize the complete upper body skeleton with both known and estimated joints.

In [ ]:
def visualize_skeleton(skeleton, title="Upper Body Skeleton"):
    """
    Visualize the complete skeleton with connections.
    """
    fig = go.Figure()
    
    joints = skeleton.get_all_joints()
    
    # Define skeleton connections (bones)
    connections = [
        ('head', 'neck'),
        ('neck', 'upper_spine'),
        ('upper_spine', 'mid_spine'),
        ('mid_spine', 'lower_spine'),
        ('upper_spine', 'left_shoulder'),
        ('upper_spine', 'right_shoulder'),
        ('left_shoulder', 'left_elbow'),
        ('left_elbow', 'left_wrist'),
        ('right_shoulder', 'right_elbow'),
        ('right_elbow', 'right_wrist'),
    ]
    
    # Plot bones as lines
    for start_joint, end_joint in connections:
        if joints[start_joint] is not None and joints[end_joint] is not None:
            start_pos = joints[start_joint]
            end_pos = joints[end_joint]
            
            fig.add_trace(go.Scatter3d(
                x=[start_pos[0], end_pos[0]],
                y=[start_pos[1], end_pos[1]],
                z=[start_pos[2], end_pos[2]],
                mode='lines',
                line=dict(color='gray', width=4),
                showlegend=False
            ))
    
    # Plot joints as markers
    for joint_name, joint_pos in joints.items():
        if joint_pos is not None:
            # Color code: red for head, blue for spine, green for arms
            if 'head' in joint_name:
                color = 'red'
                size = 12
            elif 'spine' in joint_name or 'neck' in joint_name:
                color = 'blue'
                size = 8
            else:
                color = 'green'
                size = 8
            
            fig.add_trace(go.Scatter3d(
                x=[joint_pos[0]],
                y=[joint_pos[1]],
                z=[joint_pos[2]],
                mode='markers',
                marker=dict(size=size, color=color),
                name=joint_name
            ))
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data'
        ),
        width=900,
        height=700
    )
    
    return fig

# Visualize the reconstructed skeleton
fig = visualize_skeleton(test_skeleton, f"Reconstructed Skeleton (Frame {test_frame})")
fig.show()

## 10. Single Frame Analysis

Focus on reconstructing and analyzing a single frame.

In [ ]:
# Select a frame to analyze
test_frame = 100

# Reconstruct skeleton for this single frame
print(f"Reconstructing skeleton for frame {test_frame}...")
single_skeleton = reconstruct_skeleton(
    test_frame,
    head_positions[test_frame],
    head_orientations[test_frame],
    left_wrist_pos[test_frame],
    right_wrist_pos[test_frame],
    left_forearm_pos[test_frame],
    right_forearm_pos[test_frame],
    body_model
)

print("\nReconstruction complete!")
print("\nJoint positions:")
joints = single_skeleton.get_all_joints()
for joint_name, joint_pos in joints.items():
    if joint_pos is not None:
        print(f"  {joint_name}: [{joint_pos[0]:.3f}, {joint_pos[1]:.3f}, {joint_pos[2]:.3f}]")

## 11. Validate Single Frame Results

Check bone lengths and joint positions for the reconstructed frame.

In [ ]:
def validate_skeleton(skeleton, body_model):
    """
    Validate skeleton measurements against expected bone lengths.
    """
    results = {}
    
    # Check upper arm lengths
    if skeleton.left_shoulder is not None and skeleton.left_elbow is not None:
        left_upper_arm = np.linalg.norm(skeleton.left_elbow - skeleton.left_shoulder)
        results['left_upper_arm'] = left_upper_arm
    
    if skeleton.right_shoulder is not None and skeleton.right_elbow is not None:
        right_upper_arm = np.linalg.norm(skeleton.right_elbow - skeleton.right_shoulder)
        results['right_upper_arm'] = right_upper_arm
    
    # Check shoulder width
    if skeleton.left_shoulder is not None and skeleton.right_shoulder is not None:
        shoulder_width = np.linalg.norm(skeleton.right_shoulder - skeleton.left_shoulder)
        results['shoulder_width'] = shoulder_width
    
    # Check neck length
    if skeleton.head is not None and skeleton.neck is not None:
        neck_length = np.linalg.norm(skeleton.neck - skeleton.head)
        results['neck_length'] = neck_length
    
    return results

# Validate the reconstructed skeleton
print("Validation Results for Frame", test_frame)
print("=" * 50)
print(f"\nExpected measurements (from anthropometric model):")
print(f"  Upper arm length: {body_model.upper_arm_length:.3f}m")
print(f"  Shoulder width: {body_model.shoulder_width:.3f}m")
print(f"  Neck length: {body_model.neck_length:.3f}m")

measurements = validate_skeleton(single_skeleton, body_model)
print(f"\nActual measurements:")
for key, value in measurements.items():
    print(f"  {key}: {value:.3f}m")

# Calculate errors
print(f"\nMeasurement errors:")
if 'left_upper_arm' in measurements:
    error = abs(measurements['left_upper_arm'] - body_model.upper_arm_length)
    print(f"  Left upper arm error: {error:.3f}m ({error/body_model.upper_arm_length*100:.1f}%)")
if 'right_upper_arm' in measurements:
    error = abs(measurements['right_upper_arm'] - body_model.upper_arm_length)
    print(f"  Right upper arm error: {error:.3f}m ({error/body_model.upper_arm_length*100:.1f}%)")
if 'shoulder_width' in measurements:
    error = abs(measurements['shoulder_width'] - body_model.shoulder_width)
    print(f"  Shoulder width error: {error:.3f}m ({error/body_model.shoulder_width*100:.1f}%)")
if 'neck_length' in measurements:
    error = abs(measurements['neck_length'] - body_model.neck_length)
    print(f"  Neck length error: {error:.3f}m ({error/body_model.neck_length*100:.1f}%)")

## 12. Visualize with Mediapipe (Pre-built Skeleton)

Use Mediapipe's pre-built skeleton rendering for a more realistic visualization.

In [ ]:
# Initialize Mediapipe drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles

def skeleton_to_mediapipe_landmarks(skeleton):
    """
    Convert our skeleton to Mediapipe pose landmark format.
    Mediapipe uses 33 landmarks, we'll map our joints to the relevant ones.
    """
    # Create a landmark list (Mediapipe format)
    from mediapipe.framework.formats import landmark_pb2
    
    landmarks = landmark_pb2.NormalizedLandmarkList()
    
    # Mediapipe landmark indices:
    # 0: nose, 11-12: shoulders, 13-14: elbows, 15-16: wrists
    # 23-24: hips (we'll estimate), others we'll set to 0
    
    joints = skeleton.get_all_joints()
    
    # Initialize 33 landmarks (all at origin initially)
    for i in range(33):
        landmark = landmarks.landmark.add()
        landmark.x = 0.0
        landmark.y = 0.0
        landmark.z = 0.0
        landmark.visibility = 0.0
    
    # Map our joints to Mediapipe indices
    # Note: Mediapipe expects normalized coordinates, but we'll use actual 3D positions
    # and adjust the visualization
    
    if joints['head'] is not None:
        # Nose (approximate from head)
        landmarks.landmark[0].x = joints['head'][0]
        landmarks.landmark[0].y = joints['head'][1]
        landmarks.landmark[0].z = joints['head'][2]
        landmarks.landmark[0].visibility = 1.0
    
    if joints['left_shoulder'] is not None:
        landmarks.landmark[11].x = joints['left_shoulder'][0]
        landmarks.landmark[11].y = joints['left_shoulder'][1]
        landmarks.landmark[11].z = joints['left_shoulder'][2]
        landmarks.landmark[11].visibility = 1.0
    
    if joints['right_shoulder'] is not None:
        landmarks.landmark[12].x = joints['right_shoulder'][0]
        landmarks.landmark[12].y = joints['right_shoulder'][1]
        landmarks.landmark[12].z = joints['right_shoulder'][2]
        landmarks.landmark[12].visibility = 1.0
    
    if joints['left_elbow'] is not None:
        landmarks.landmark[13].x = joints['left_elbow'][0]
        landmarks.landmark[13].y = joints['left_elbow'][1]
        landmarks.landmark[13].z = joints['left_elbow'][2]
        landmarks.landmark[13].visibility = 1.0
    
    if joints['right_elbow'] is not None:
        landmarks.landmark[14].x = joints['right_elbow'][0]
        landmarks.landmark[14].y = joints['right_elbow'][1]
        landmarks.landmark[14].z = joints['right_elbow'][2]
        landmarks.landmark[14].visibility = 1.0
    
    if joints['left_wrist'] is not None:
        landmarks.landmark[15].x = joints['left_wrist'][0]
        landmarks.landmark[15].y = joints['left_wrist'][1]
        landmarks.landmark[15].z = joints['left_wrist'][2]
        landmarks.landmark[15].visibility = 1.0
    
    if joints['right_wrist'] is not None:
        landmarks.landmark[16].x = joints['right_wrist'][0]
        landmarks.landmark[16].y = joints['right_wrist'][1]
        landmarks.landmark[16].z = joints['right_wrist'][2]
        landmarks.landmark[16].visibility = 1.0
    
    # Estimate hip positions (for better skeleton visualization)
    if joints['lower_spine'] is not None:
        # Place hips slightly to the sides of lower spine
        hip_offset = 0.1  # 10cm to each side
        landmarks.landmark[23].x = joints['lower_spine'][0] - hip_offset
        landmarks.landmark[23].y = joints['lower_spine'][1]
        landmarks.landmark[23].z = joints['lower_spine'][2]
        landmarks.landmark[23].visibility = 1.0
        
        landmarks.landmark[24].x = joints['lower_spine'][0] + hip_offset
        landmarks.landmark[24].y = joints['lower_spine'][1]
        landmarks.landmark[24].z = joints['lower_spine'][2]
        landmarks.landmark[24].visibility = 1.0
    
    return landmarks

# Convert our skeleton to Mediapipe format
mp_landmarks = skeleton_to_mediapipe_landmarks(single_skeleton)
print("Converted skeleton to Mediapipe landmark format")
print(f"Total landmarks: {len(mp_landmarks.landmark)}")
print(f"Visible landmarks: {sum(1 for lm in mp_landmarks.landmark if lm.visibility > 0)}")

In [ ]:
def visualize_with_mediapipe_3d(landmarks, title="Mediapipe Skeleton"):
    """
    Create a 3D visualization using Mediapipe's pose connections.
    """
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Extract landmark positions
    points = []
    for lm in landmarks.landmark:
        if lm.visibility > 0:
            points.append([lm.x, lm.y, lm.z])
    
    # Mediapipe pose connections (upper body only)
    connections = [
        (11, 12),  # Shoulders
        (11, 13),  # Left shoulder to elbow
        (13, 15),  # Left elbow to wrist
        (12, 14),  # Right shoulder to elbow
        (14, 16),  # Right elbow to wrist
        (11, 23),  # Left shoulder to hip
        (12, 24),  # Right shoulder to hip
        (23, 24),  # Hips
    ]
    
    # Draw connections
    for connection in connections:
        start_idx, end_idx = connection
        start_lm = landmarks.landmark[start_idx]
        end_lm = landmarks.landmark[end_idx]
        
        if start_lm.visibility > 0 and end_lm.visibility > 0:
            ax.plot(
                [start_lm.x, end_lm.x],
                [start_lm.y, end_lm.y],
                [start_lm.z, end_lm.z],
                'b-', linewidth=2
            )
    
    # Draw landmarks
    for i, lm in enumerate(landmarks.landmark):
        if lm.visibility > 0:
            color = 'red' if i == 0 else 'green'  # Red for head, green for others
            ax.scatter(lm.x, lm.y, lm.z, c=color, s=50, alpha=0.8)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(title)
    
    # Set equal aspect ratio
    joints = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark if lm.visibility > 0])
    max_range = np.array([
        joints[:, 0].max() - joints[:, 0].min(),
        joints[:, 1].max() - joints[:, 1].min(),
        joints[:, 2].max() - joints[:, 2].min()
    ]).max() / 2.0
    
    mid_x = (joints[:, 0].max() + joints[:, 0].min()) * 0.5
    mid_y = (joints[:, 1].max() + joints[:, 1].min()) * 0.5
    mid_z = (joints[:, 2].max() + joints[:, 2].min()) * 0.5
    
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.tight_layout()
    plt.show()

# Visualize with Mediapipe
visualize_with_mediapipe_3d(mp_landmarks, f"Mediapipe Skeleton - Frame {test_frame}")

### Compare Plotly vs Mediapipe Visualization

Let's see both visualizations side by side.

In [ ]:
# Show both visualizations
print("=" * 60)
print("PLOTLY VISUALIZATION (Custom)")
print("=" * 60)
fig_plotly = visualize_skeleton(single_skeleton, f"Plotly - Frame {test_frame}")
fig_plotly.show()

print("\n" + "=" * 60)
print("MEDIAPIPE VISUALIZATION (Pre-built)")
print("=" * 60)
visualize_with_mediapipe_3d(mp_landmarks, f"Mediapipe - Frame {test_frame}")

print("\nBoth visualizations complete!")